In [ ]:
!pip install nltk
!pip install -q wordcloud
!pip install matplotlib
!pip install vaderSentiment
!pip install NRCLex
!pip install seaborn
!pip install tensorflow
!pip install keras
!pip install scikit-learn

# Import the Dataset

In [ ]:
from nrclex import NRCLex
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from keras.src.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.src.utils import pad_sequences
from keras.models import Sequential
import pandas as pd
from nltk.corpus import stopwords
import seaborn as sb
import matplotlib as plt
from pandas import DataFrame as df
import matplotlib.pyplot as mp




In [ ]:
df['Message Size'] = df['text'].apply(lambda x: len(x))

In [ ]:
df.head(10)

In [ ]:
df1 = df.copy();

df1.drop(columns=['text'], inplace=True)

plt.figure(figsize=(15, 15))
# plotting correlation heatmap 
dataplot = sb.heatmap(df1.corr(), cmap="YlGnBu", annot=True)

# displaying heatmap 
mp.show()

In [234]:
stop = stopwords.words('english')

df = pd.read_csv("./datasets/train2Data.csv", encoding="utf-8", sep=',')

df.rename(columns={'Tweets': 'text', ' Label': 'label'}, inplace=True)

#Remove stopwords from the dataset
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['text'] = df['text'].apply(lambda x: ' '.join([item for item in x.split() if item not in ['amp', 'quot']]))

analyzer = SentimentIntensityAnalyzer()

print("Getting Fear")
df['fear'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('fear', 0))

print("Getting Anger")
df['anger'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('anger', 0))

print("Getting Anticipation")
df['anticipation'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('anticipation', 0))

print("Getting Trust")
df['trust'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('trust', 0))

print("Getting Surprise")
df['surprise'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('surprise', 0))

print("Getting Positive")
df['positive'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('positive', 0))

print("Getting Negative")
df['negative'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('negative', 0))

print("Getting Sadness")
df['sadness'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('sadness', 0))

print("Getting Disgust")
df['disgust'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('disgust', 0))

print("Getting Joy")
df['joy'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('joy', 0))

for index, row in df.iterrows():
    sentence = row['text']
    sentiment = analyzer.polarity_scores(sentence)

    df.at[index, 'negative'] = sentiment['neg']
    df.at[index, 'neutral'] = sentiment['neu']
    df.at[index, 'positive'] = sentiment['pos']
    df.at[index, 'compound'] = sentiment['compound']

Getting Fear
Getting Anger
Getting Anticipation
Getting Trust
Getting Surprise
Getting Positive
Getting Negative
Getting Sadness
Getting Disgust
Getting Joy


In [255]:
X_attr = df.loc[:, df.columns != 'text']
X_attr = X_attr.drop('label', axis=1)
X_attr.head()

X_train, X_test, X_train_attr, X_test_attr, Y_train, Y_test = train_test_split(X_text, X_attr, df['label'],
                                                                               test_size=0.20, random_state=42)

,fear,anger,anticipation,trust,surprise,positive,negative,sadness,disgust,joy,neutral,compound
0,0,1,0,0,0,0.000,0.692,1,1,0,0.308,-0.8402
1,1,2,2,1,1,0.208,0.324,2,2,1,0.468,-0.6387
2,0,1,1,0,0,0.000,0.488,1,1,0,0.512,-0.7703
3,0,2,0,0,0,0.000,0.813,1,1,0,0.187,-0.9198
4,1,3,0,0,0,0.216,0.483,3,3,0,0.302,-0.7351


# Basic Sequential Model

In [262]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['text'].values)
X_text = tokenizer.texts_to_sequences(df['text'].values)
X_text = pad_sequences(X_text)

embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim, input_length=X_text.shape[1]))
model.add(Dense(16))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, Y_train, epochs=7, validation_data=(X_test, Y_test), batch_size=32, verbose=1)


Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_52 (Embedding)    (None, 36, 128)           256000    
                                                                 
 dense_75 (Dense)            (None, 36, 16)            2064      
                                                                 
 dense_76 (Dense)            (None, 36, 1)             17        
                                                                 
Total params: 258081 (1008.13 KB)
Trainable params: 258081 (1008.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/7
635/775 [=======================>......] - ETA: 0s - loss: 0.6432 - accuracy: 0.5910

KeyboardInterrupt: 

In [295]:
from keras.src.layers import concatenate
from keras.layers import Dense, Embedding, LSTM, Bidirectional, Input
from keras.models import Model


inputText = Input(shape=(2000,))
inputAttr = Input(shape=(128,))

x = Embedding(max_fatures, embed_dim, input_length=X_text.shape[1])(inputText)
x = Bidirectional(LSTM(64, return_sequences=True))(x)
x = Model(inputs=inputText, outputs=x);


combined = concatenate([x.output, inputAttr])

z = Dense(1)(combined)

model = Model(inputs=[x.input, y], outputs=z)

model.summary()





ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 2000, 128), (None, 128)]

In [334]:
from keras.src.layers import concatenate, Dropout
from keras.layers import Dense, Embedding, LSTM, Bidirectional, Input
from keras.models import Model

word_index = 300
max_length = 2000

# Defining the embedding layer
embedding_dim = 64
input1 = Input(shape=(X_text.shape[1],))

embedding_layer = Embedding(max_fatures, embed_dim, input_length=X_text.shape[1])(input1)

# Building LSTM for text features                          
x = Bidirectional(LSTM(embedding_dim, return_sequences=True))(embedding_layer)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(embedding_dim))(x)

x = Model(inputs=input1, outputs=x)

#Inputting Number features
input2 = Input(shape=(X_attr.shape[1],))

# Merging inputs
merge = concatenate([x.output, input2])

# Building dense layers for regression with number features
output = Dense(1, activation='sigmoid')(merge)


model = Model(inputs=[x.input, input2], outputs=output)

model.compile(optimizer="adam", loss='binary_crossentropy', metrics='binary_accuracy')

In [332]:
model.summary()

Model: "model_81"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_98 (InputLayer)       [(None, 36)]                 0         []                            
                                                                                                  
 embedding_99 (Embedding)    (None, 36, 128)              256000    ['input_98[0][0]']            
                                                                                                  
 bidirectional_55 (Bidirect  (None, 36, 128)              98816     ['embedding_99[0][0]']        
 ional)                                                                                           
                                                                                                  
 dropout_1 (Dropout)         (None, 36, 128)              0         ['bidirectional_55[0][0

In [323]:
X_train.shape

(24800, 36)

In [335]:
H = model.fit(x=[X_train, X_train_attr], y=Y_train, validation_data=([X_test, X_test_attr], Y_test), epochs=10, verbose=1, batch_size=16)

Epoch 1/10
1550/1550 [==============================] - 59s 33ms/step - loss: 0.0491 - binary_accuracy: 0.9788 - val_loss: 0.0154 - val_binary_accuracy: 0.9969
Epoch 2/10
 113/1550 [=>............................] - ETA: 41s - loss: 0.0041 - binary_accuracy: 0.9989

KeyboardInterrupt: 